### Comparing methods


In [2]:
import numpy as np 
import importlib
import testDijkToEdge
import maxProbAstar
import path_evaluation
from maxProbAstar import MaxProbAstar
from problem_setting import ProblemSetting
from path_evaluation import PathEvaluation
import copy
import math
import matplotlib.pyplot as plt
import time
import automaton_dp
import plot3d_fire
import pickle
import seaborn as sns
import os
import functools
from testDijkToEdge import UpdateEnv

In [168]:
# problem parameters
_SIZE = 20
_STPFlag = 1 # flag for enabling STP. If set to 0, use average probability
_obsvEffectRange=2
_replanRange=2

In [4]:
# problem setup
xTar = 8
yTar = 16
xEnd = 11
yEnd = 19
p1 = ProblemSetting(target = [[xTar,yTar]], endPoint = [[xEnd,yEnd]],_stochastic_environment_flag=1, _setting_num=1)
# monteCarloAverageFireMap, monteCarloFireMap = p1.compute_monteCarlo_2(3000)
monteCarloFireMap = pickle.load(open('MCFMs3000',"rb"))
monteCarloAverageFireMap = pickle.load(open('MCAFMs3000',"rb"))

In [194]:
# DP solution
# aDP1 = automaton_dp.AutomatonDP(p1.M,p1.N,p1.T,p1.Map,monteCarloAverageFireMap, monteCarloFireMap, p1.StartPoint,p1.EndPoint,p1.Target,p1.Wall,p1.Fire,Q=4, STPFlag=_STPFlag)
# aDP1.solve()
# with open('aDP1','wb') as fp:
#         pickle.dump(aDP1, fp)
aDP1 = pickle.load(open('aDP1',"rb"))

In [278]:
# reload stuff
importlib.reload(path_evaluation)
from path_evaluation import PathEvaluation
importlib.reload(maxProbAstar)
from maxProbAstar import MaxProbAstar
importlib.reload(testDijkToEdge)
from testDijkToEdge import UpdateEnv

# let the robot observe at xT=8,yT=7, at tT = 7
tT = 7; xT = 8; yT = 7; # s=138 for (7,9), 7 for (7,7) # 653 for lots
upe = UpdateEnv(p1.M,p1.N,p1.Wall,p1.Fire,monteCarloFireMap, monteCarloAverageFireMap, obsvEffectRange=_obsvEffectRange, replanRange=_replanRange,tT = tT,aDP1=aDP1)


T = tT
f = upe.observe(1243,T,xT,yT)
print(f)

[[6 6]]


In [217]:
 _t = time.time()
upe.dijk_to_edge(upe.replanRange)
elapsed_ = time.time() - _t
print('Replanning took: %f seconds' %(elapsed_))

[(8, 7), (7, 7), (6, 7)]
[(8, 7), (8, 6), (7, 6)]
[(8, 7), (8, 8), (7, 8)]
[(8, 7), (8, 6), (8, 5)]
[(8, 7), (8, 8), (8, 9)]
[(8, 7), (9, 7), (9, 6)]
[(8, 7), (9, 7), (9, 8)]
[(8, 7), (9, 7), (10, 7)]
Replanning took: 44.488209 seconds


In [222]:
maxNeighborIndex = np.argmax(np.exp(np.multiply(-1,upe.fullPossibleProbs)))
newPath = copy.deepcopy(upe.possiblePathsAfter[maxNeighborIndex])
for p in range(len(upe.possiblePathsBefore[maxNeighborIndex])-2,-1,-1):
    newPath.insert(0,(upe.possiblePathsBefore[maxNeighborIndex][p][0],upe.possiblePathsBefore[maxNeighborIndex][p][1],-1)) # NOTICE: only -1 for now

In [228]:
# another MC samples. Wrongly generated that...
# p2 = ProblemSetting(target = [[xTar,yTar]], endPoint = [[xEnd,yEnd]],_stochastic_environment_flag=1, _setting_num=1)
# monteCarloAverageFireMapExam, monteCarloFireMapExam = p2.compute_monteCarlo_2(3000,pickleSave=False)
monteCarloFireMapExam = pickle.load(open('MCFMsExam3000',"rb"))
monteCarloAverageFireMapExam = pickle.load(open('MCAFMsExam3000',"rb"))

In [236]:
newPath

[(8, 7, -1),
 (9, 7, -1),
 (10, 7, -1),
 (11, 7, -1),
 (12, 7, -1),
 (13, 7, -1),
 (14, 7, -1),
 (14, 8, -1),
 (14, 9, -1),
 (14, 10, -1),
 (14, 11, -1),
 (14, 12, -1),
 (14, 13, -1),
 (14, 14, -1),
 (14, 15, -1),
 (14, 16, -1),
 (13, 16, -1),
 (12, 16, -1),
 (11, 16, -1),
 (10, 16, -1),
 (9, 16, -1),
 (8, 16, -2),
 (8, 17, -2),
 (8, 18, -2),
 (8, 19, -2),
 (9, 19, -2),
 (10, 19, -2),
 (11, 19, -3)]

In [237]:
pe2 = PathEvaluation(monteCarloFireMapExam)
pe2.evaluate_path_t(newPath, tT)

0.13466666666666666

In [238]:
aDP1.pe.evaluate_path_t(newPath, tT)

0.11933333333333333

In [261]:
import problem_setting
importlib.reload(problem_setting)
from problem_setting import ProblemSetting

In [263]:
p3 = ProblemSetting(target = [[xTar,yTar]], endPoint = [[xEnd,yEnd]],_stochastic_environment_flag=1, _setting_num=1)
monteCarloAverageFireMapExamObsv, monteCarloFireMapExamObsv = p3.compute_monteCarlo_obsv(500,pickleSave=True,fireLocationArray=f,obsvTime=tT)

counter is 5095
h is 500


In [264]:
pe3 = PathEvaluation(monteCarloFireMapExamObsv)
pe3.evaluate_path_t(newPath, tT)

0.132

Mass experiments

In [283]:
sSet = [134,138,144,154,156,158,168,202,211,218]
newPathList = []
oldSafeProbList = []
newSafeProbList = []
upe = UpdateEnv(p1.M,p1.N,p1.Wall,p1.Fire,monteCarloFireMap, monteCarloAverageFireMap, obsvEffectRange=_obsvEffectRange, replanRange=_replanRange,tT = tT,aDP1=aDP1)
for s in sSet:   
    f = upe.observe(s,T,xT,yT)
    _t = time.time()
    upe.dijk_to_edge(upe.replanRange)
    maxNeighborIndex = np.argmax(np.exp(np.multiply(-1,upe.fullPossibleProbs)))
    newPath = copy.deepcopy(upe.possiblePathsAfter[maxNeighborIndex])
    for p in range(len(upe.possiblePathsBefore[maxNeighborIndex])-2,-1,-1):
        newPath.insert(0,(upe.possiblePathsBefore[maxNeighborIndex][p][0],upe.possiblePathsBefore[maxNeighborIndex][p][1],-1)) # NOTICE: only -1 for now
    newPathList.append(newPath)
    elapsed_ = time.time() - _t
    print('Replanning took: %f seconds' %(elapsed_))
    oldSafeProbList.append(aDP1.pe.evaluate_path_t(newPath, tT))
    monteCarloAverageFireMapExamObsv, monteCarloFireMapExamObsv = p3.compute_monteCarlo_obsv(500,pickleSave=False,fireLocationArray=f,obsvTime=tT)
    pe3 = PathEvaluation(monteCarloFireMapExamObsv)
    newSafeProbList.append(pe3.evaluate_path_t(newPath, tT))

Replanning took: 44.778396 seconds
counter is 6160
h is 500
Replanning took: 41.518188 seconds
counter is 5266
h is 500
Replanning took: 37.484935 seconds
counter is 5944
h is 500
Replanning took: 36.781745 seconds
counter is 57984
h is 500
Replanning took: 34.731144 seconds
counter is 58501
h is 500
math domain error
math domain error
Replanning took: 33.835026 seconds
counter is 52739
h is 500
Replanning took: 37.439872 seconds
counter is 15238
h is 500
Replanning took: 35.108138 seconds
counter is 15798
h is 500
Replanning took: 34.905913 seconds
counter is 26071
h is 500
Replanning took: 43.894645 seconds
counter is 4740
h is 500


In [284]:
print(oldSafeProbList)
print(newSafeProbList)

[0.146, 0.11933333333333333, 0.11933333333333333, 0.11933333333333333, 0.060333333333333336, 0.11933333333333333, 0.11933333333333333, 0.11933333333333333, 0.11933333333333333, 0.146]
[0.116, 0.132, 0.146, 0.09, 0.044, 0.134, 0.066, 0.112, 0.05, 0.068]


In [289]:
oldCost, oldPath, oldControl = aDP1.query_optimal_solution(0, [8,0,-1])
aDP1.remove_duplicated_path_segs(oldPath)
print(oldPath)

[(8, 0, -1), (8, 1, -1), (8, 2, -1), (8, 3, -1), (8, 4, -1), (8, 5, -1), (8, 6, -1), (8, 7, -1), (7, 7, -1), (7, 8, -1), (6, 8, -1), (6, 9, -1), (6, 10, -1), (5, 10, -1), (5, 11, -1), (5, 12, -1), (5, 13, -1), (5, 14, -1), (5, 15, -1), (5, 16, -1), (6, 16, -1), (7, 16, -1), (8, 16, -2), (8, 17, -2), (8, 18, -2), (8, 19, -2), (9, 19, -2), (10, 19, -2), (11, 19, -3)]


In [290]:
pe3.evaluate_path_t(oldPath[7:], tT)

0.068

## Test code

In [211]:
routesFromNowCopied = copy.deepcopy(routesFromNow)
upe.remove_duplicated_path_segs(routesFromNowCopied)
routesFromNowCopied

[(7, 7, -1),
 (7, 8, -1),
 (6, 8, -1),
 (6, 9, -1),
 (5, 9, -1),
 (5, 10, -1),
 (5, 11, -1),
 (5, 12, -1),
 (5, 13, -1),
 (5, 14, -1),
 (5, 15, -1),
 (5, 16, -1),
 (6, 16, -1),
 (7, 16, -1),
 (8, 16, -2),
 (8, 17, -2),
 (8, 18, -2),
 (8, 19, -2),
 (9, 19, -2),
 (10, 19, -2),
 (11, 19, -3)]

In [215]:
safeProbSegmentList2 = [upe.pe.evaluate_segment_obsv(9+ind,
                               [routesFromNow[ind][0],
                                routesFromNow[ind][1]],
                               9+ind+1,
                               [routesFromNow[ind+1][0],
                                routesFromNow[ind+1][1]],
                               9,
                               upe.observedFireLocationArray) 
 for ind,(x,y,q) in enumerate(routesFromNowCopied[:-1])]
safeProbSegmentList2

[0.8473581213307241,
 0.6096579476861167,
 0.6948640483383686,
 0.6109589041095891,
 0.7465753424657534,
 0.7075471698113207,
 0.6965944272445821,
 0.7763578274760383,
 0.8532934131736527,
 0.9609756097560975,
 0.9487179487179487,
 0.8686679174484052,
 0.8486707566462167,
 0.825,
 0.9405940594059405,
 0.9501084598698482,
 0.9780439121756487,
 0.9147005444646098,
 0.9123809523809524,
 0.9278752436647173]

In [174]:
upe.dijkPathFinder.savedSafeProb[7,8,7,8,6]

0.973421926910299

In [193]:
costsFromNow, routesFromNow, controlsFromNow = aDP1.query_optimal_solution(9, [7,7,-1])
routesFromNow

[(7, 7, -1),
 (7, 8, -1),
 (6, 8, -1),
 (6, 9, -1),
 (5, 9, -1),
 (5, 10, -1),
 (5, 11, -1),
 (5, 12, -1),
 (5, 13, -1),
 (5, 14, -1),
 (5, 15, -1),
 (5, 16, -1),
 (6, 16, -1),
 (7, 16, -1),
 (8, 16, -2),
 (8, 17, -2),
 (8, 18, -2),
 (8, 19, -2),
 (9, 19, -2),
 (10, 19, -2),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3),
 (11, 19, -3)]

In [123]:
_pathSegment, searchNodes, lastNode = MaxProbAstar(upe.M,upe.N,upe.Wall,monteCarloFireMap,startTime = upe.T, xT=xT,yT=yT,observedFireLocationArray = upe.observedFireLocationArray).astar(tuple([upe.xT,upe.yT]),tuple(loc2))

Corresponding episode number: 302
safeProb from (8, 7) to (8, 6) is 0.973421926910299
Corresponding episode number: 302
safeProb from (8, 7) to (8, 8) is 0.946843853820598
Corresponding episode number: 302
safeProb from (8, 7) to (7, 7) is 0.8039867109634552
Corresponding episode number: 302
safeProb from (8, 7) to (9, 7) is 0.9966777408637874
Corresponding episode number: 302
safeProb from (9, 7) to (9, 6) is 1.0
Corresponding episode number: 302
safeProb from (9, 7) to (9, 8) is 0.9169435215946844
Corresponding episode number: 302
safeProb from (9, 7) to (10, 7) is 0.9966777408637874
Corresponding episode number: 302
safeProb from (9, 6) to (9, 5) is 0.9900662251655629
Corresponding episode number: 302
safeProb from (9, 6) to (8, 6) is 0.8774834437086093
Corresponding episode number: 302
safeProb from (9, 6) to (10, 6) is 0.9966887417218543
Corresponding episode number: 302
safeProb from (10, 6) to (10, 5) is 0.9966777408637874
Corresponding episode number: 302
safeProb from (10, 6) 

Corresponding episode number: 302
safeProb from (17, 0) to (18, 0) is 1.0
Corresponding episode number: 302
safeProb from (15, 2) to (15, 1) is 1.0
Corresponding episode number: 302
safeProb from (15, 2) to (15, 3) is 1.0
Corresponding episode number: 302
safeProb from (15, 2) to (14, 2) is 1.0
Corresponding episode number: 302
safeProb from (19, 2) to (19, 1) is 1.0
Corresponding episode number: 302
safeProb from (19, 2) to (18, 2) is 1.0
Corresponding episode number: 302
safeProb from (18, 1) to (18, 0) is 1.0
Corresponding episode number: 302
safeProb from (18, 1) to (18, 2) is 1.0
Corresponding episode number: 302
safeProb from (18, 1) to (19, 1) is 1.0
Corresponding episode number: 302
safeProb from (18, 0) to (19, 0) is 1.0
Corresponding episode number: 302
safeProb from (16, 1) to (15, 1) is 1.0
Corresponding episode number: 302
safeProb from (19, 1) to (19, 0) is 1.0
Corresponding episode number: 302
safeProb from (14, 2) to (14, 3) is 1.0
Corresponding episode number: 302
safe

Corresponding episode number: 302
safeProb from (19, 11) to (19, 12) is 0.9831081081081081
Corresponding episode number: 302
safeProb from (19, 11) to (18, 11) is 0.9695945945945946
Corresponding episode number: 302
safeProb from (18, 10) to (18, 9) is 1.0
Corresponding episode number: 302
safeProb from (18, 10) to (18, 11) is 1.0
Corresponding episode number: 302
safeProb from (18, 11) to (18, 12) is 0.9933774834437086
Corresponding episode number: 302
safeProb from (18, 12) to (18, 13) is 1.0
Corresponding episode number: 302
safeProb from (18, 12) to (17, 12) is 0.9933333333333333
Corresponding episode number: 302
safeProb from (18, 12) to (19, 12) is 1.0
Corresponding episode number: 302
safeProb from (18, 13) to (18, 14) is 0.9966777408637874
Corresponding episode number: 302
safeProb from (18, 13) to (17, 13) is 0.9867109634551495
Corresponding episode number: 302
safeProb from (18, 13) to (19, 13) is 1.0
Corresponding episode number: 302
safeProb from (7, 0) to (7, 1) is 0.91349

Corresponding episode number: 302
safeProb from (7, 3) to (7, 4) is 0.7316176470588235
Corresponding episode number: 302
safeProb from (7, 3) to (6, 3) is 0.9411764705882353
Corresponding episode number: 302
safeProb from (7, 4) to (7, 5) is 0.5641025641025641
Corresponding episode number: 302
safeProb from (7, 4) to (6, 4) is 0.9487179487179487
Corresponding episode number: 302
safeProb from (15, 10) to (15, 11) is 0.9176029962546817
Corresponding episode number: 302
safeProb from (15, 10) to (14, 10) is 0.7602996254681648
Corresponding episode number: 302
safeProb from (14, 9) to (14, 10) is 0.8787878787878788
Corresponding episode number: 302
safeProb from (14, 9) to (13, 9) is 0.7992424242424242
Corresponding episode number: 302
safeProb from (16, 19) to (16, 18) is 0.8407407407407408
Corresponding episode number: 302
safeProb from (16, 19) to (15, 19) is 0.9037037037037037
Corresponding episode number: 302
safeProb from (7, 8) to (7, 7) is 0.7096774193548387
Corresponding episode 

In [124]:
pathSegment = list(_pathSegment)

In [125]:
pathSegment

[(8, 7), (8, 8), (7, 8), (6, 8)]

In [112]:
[[pathSegment[ind+1][0],pathSegment[ind+1][1]] for ind,(x,y) in enumerate(pathSegment[:-1])]

[[8, 8], [7, 8], [6, 8]]

In [126]:
safeProbSegmentList = [upe.pe.evaluate_segment_obsv(upe.T+ind,[pathSegment[ind][0],pathSegment[ind][1]],upe.T+ind+1,[pathSegment[ind+1][0],pathSegment[ind+1][1]],upe.T,upe.observedFireLocationArray) for ind,(x,y) in enumerate(pathSegment[:-1])]
print(safeProbSegmentList)

Corresponding episode number: 302
safeProb from [8, 7] to [8, 8] is 0.946843853820598
Corresponding episode number: 302
safeProb from [8, 8] to [7, 8] is 0.856140350877193
Corresponding episode number: 302
safeProb from [7, 8] to [6, 8] is 0.6532258064516129
[0.946843853820598, 0.856140350877193, 0.6532258064516129]


In [129]:
safeProbSegment = functools.reduce(lambda a,b:a*b, safeProbSegmentList)
print(safeProbSegment)
-math.log(safeProbSegment)

0.5295252384524702


0.635774450388272